### Training NLP models in Colab without running out of RAM

This notebook focuses on some techniques you can use to avoid running out of memory, when working with a lot of data and large models used for NLP tasks.

The task we'll work on is textual entailment, using the [Stanford Natural Language Inference (SNLI) dataset](https://nlp.stanford.edu/projects/snli/). We'll build a fairly simple classification model, using a pre-trained BERT model. (Some of the code is inspired by [this Keras example for SNLI classification](https://keras.io/examples/nlp/semantic_similarity_with_bert/).)

The main focus of this notebook is not on the task or model architecture, but on how to load part of your data at a time while you train, and save model checkpoints as you go. You should be able to run the notebook on the free tier of Google Colab. (There is a point where it will run out of RAM, for demonstration, but that is noted in the comments.)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2024-summer-main/blob/master/materials/walkthrough_notebooks/keras_with_limited_ram/keras_training_with_limited_ram.ipynb)


In [1]:
!pip install -q transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.6 MB/s eta 0:00:00


In [2]:
import os, re
import time
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# These auto classes load the right type of tokenizer and model based on a model name
from transformers import BertTokenizer, TFBertModel

We'll start by downloading the data, using curl in bash to save it to the local disk space for the Colab notebook. You might have your data in Google Drive instead; later we'll mount a Drive folder to this notebook so that we can save our model someplace more permanent, but you can move that step up if you need to load data from Drive.

In [3]:
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  7250k      0  0:00:01  0:00:01 --:--:-- 7246k
SNLI_Corpus/
SNLI_Corpus/snli_1.0_dev.csv
SNLI_Corpus/snli_1.0_train.csv
SNLI_Corpus/snli_1.0_test.csv


In [4]:
!ls SNLI_Corpus

snli_1.0_dev.csv  snli_1.0_test.csv  snli_1.0_train.csv


First let's read in the entire train and dev datasets. It looks like we have about 550k training examples, and 10k dev examples (which we'll use for validation). Just loading those short sentence pairs doesn't take a lot of RAM, but it will be too much to process with a BERT model. (You can see how much RAM and Disk space you're using by looking in the upper right corner of the notebook.)

In [5]:
train_filename = 'SNLI_Corpus/snli_1.0_train.csv'
dev_filename = 'SNLI_Corpus/snli_1.0_dev.csv'

df_train = pd.read_csv(train_filename)
df_dev = pd.read_csv(dev_filename)

df_train.shape, df_dev.shape

((550152, 3), (10000, 3))

Let's define some functions that we'll need to preprocess the data and build our classification model. First, we'll tokenize the sentence pairs using the pretrained BERT tokenizer. Second, we need to convert the three label classes from strings to numeric values.

In [6]:
label_dict = {'neutral': 0, 'entailment': 1, 'contradiction': 2}

In [7]:
def preprocess_data(sentence_pairs, label_strs, tokenizer, max_length=128):
    # With BERT tokenizer's batch_encode_plus, sentence pairs are
    # encoded together and separated by [SEP] token.
    encoded = tokenizer.batch_encode_plus(
        sentence_pairs,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors="tf"
    )

    # Extract encoded features and labels, add to corresponding lists
    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")
    token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

    # Convert string labels into numbered categories
    labels = np.array([label_dict[label] if label in label_dict else 0
                       for label in label_strs])

    return [input_ids, attention_masks, token_type_ids], labels

For the model, we'll construct a fairly simple classification model on top of the pretrained BERT model. Since we're freezing the full BERT model, it doesn't work very well for this classification problem to just use the pre-trained CLS token output as our vector representing the full input that we want to classify. (It would probably work better if we unfroze some BERT layers to fine-tune that CLS token.) Instead, we'll add one more attention layer on top of the full sequence of contextualized token vectors that we get out of BERT, so that we can train that attention layer to pay attention to the tokens that are most useful for this entailment task.

In [28]:
def build_snli_model(bert_model, max_length=128, hidden_dim=256):
    # Freeze all layers of the BERT model except the last layer
    for w in bert_model.weights:
        if not 'layer_._11' in w.name:
            w._trainable = False

    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_masks')
    token_type_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='token_type_ids')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_output = bert_model(bert_inputs)
    cls_output = bert_output[0][:, 0, :]

    dropout_output = layers.Dropout(0.3)(cls_output)
    final_output = layers.Dense(3, activation="softmax")(dropout_output)

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, token_type_ids],
                                  outputs=[final_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

Ok, let's load the pretrained tokenizer and model, build the classification model, and preprocess our data to get ready to train. We'll freeze the BERT model layers in this example, so we keep the pre-trained weights rather than fine-tuning. (We will still be training the new layers we add on top of BERT for classification.) If you set the last line in the cell below to True, and train further on your task, you'll be fine-tuning the BERT model. It will take longer to train.

In [29]:
bert_model_name='bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a 

In [30]:
model = build_snli_model(bert_model, max_length=128, hidden_dim=256)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_masks (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 token_type_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                            

Here's the code to run our preprocess_data function (tokenizing the text into vocab_ids) on the dev set and then the full train set. Note that your notebook might crash if you try to tokenize the full dataset, so only run that code if you want to try and see what happens.

In [31]:
# Tokenize the 1k of the dev examples to use for validation data first
# (You can use more, 1k works well for a quick example)

dev_sentence_pairs = df_dev[['sentence1', 'sentence2']].values[:1000].astype(str).tolist()
dev_labels = df_dev['similarity'].values[:1000]

dev_data = preprocess_data(
    dev_sentence_pairs, dev_labels, tokenizer=bert_tokenizer, max_length=128
)

In [ ]:
# Now tokenize the 550k training examples ...
# ONLY RUN THIS CELL THE FIRST TIME FOR DEMONSTRATION, IT MIGHT RUN OUT OF RAM

sentence_pairs = df_train[['sentence1', 'sentence2']].values.astype(str).tolist()
labels = df_train['similarity'].values

train_data = preprocess_data(
    sentence_pairs, labels, tokenizer=bert_tokenizer, max_length=128
)

At some point when running the last cell, if you're using the free Collab tier, your notebook probably ran out of RAM and crashed. (If it didn't, you're on a Colab machine with more RAM; available resources may vary. But you may not be able to actually train the model with all of that data in memory.)

Let's try again, but this time, we won't load all of our data at once. Connect the notebook again (it may have restarted on its own), and run most of the code above, but stop after tokenizing the dev data (which we'll keep for validation below). Don't tokenize the full 550k dataset.

In [33]:
# In case you loaded the full dataset above
df_train = None

We can define a [custom class called a data generator](https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3), that we will pass to model.fit instead of our full dataset. This data generator needs to implement methods for `__len__`, `__getitem__`, and `on_epoch_end`. In `__getitem__`, we'll write the code to get the next batch of data to train the model. We can write that function so it only loads and tokenizes the data needed for the next batch. In `on_epoch_end` we'll shuffle the order in which we plan to load data for the next epoch.

We'll look at how to do this two ways. First, in the data we downloaded from SNLI, all of the training data is in one large CSV file. We can use the pandas `pd.read_csv` method, which includes options to skip certain rows of data and only load a certain number. We won't just want to load a consecutive chunk each time, because we'll want to shuffle the rows. The `read_csv` method doesn't quite have an option to specify individual row indices to load, but we can specify a list of row indices to skip, so that's what we'll do here.

In [34]:
class SNLIDataGeneratorFromFile(tf.keras.utils.Sequence):

    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 batch_size=32,
                 shuffle=True):

        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()

    def __len__(self):
        # Return the number of batches in the full dataset
        # return self.n_examples // self.batch_size
        return 100

    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this batch
        batch_idx_skip = self.row_order[:batch_start] + self.row_order[batch_end:]
        df = pd.read_csv(self.data_filename, skiprows=batch_idx_skip)

        sentence_pairs = df[['sentence1', 'sentence2']].values.astype(str).tolist()
        labels = df['similarity'].values

        batch_data = preprocess_data(
            sentence_pairs,
            labels,
            self.tokenizer,
            self.max_length
        )

        return batch_data

    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [35]:
# Create an instance of our data generator, for our training data file and size

train_data_generator = SNLIDataGeneratorFromFile(
    tokenizer=bert_tokenizer,
    n_examples=550152,
    data_filename='SNLI_Corpus/snli_1.0_train.csv'
)

One more thing. It's going to take a while to train our model (even with a GPU, which we'll need to use). Colab resources are free but shared, there are usage limits and our notebook might time out especially when using a GPU for a while. So we should periodically save a copy of our trained model as we go. Later, we can load the model that we saved and keep training it further.

At this point, we probably do want to mount a Google Drive folder, because we won't want to save our checkpoints just to temporary Colab disk space. If our notebook disconnects, we'll lose those files. The next cell mounts your Drive folder, then for demonstration I'm showing my (the instructor's) UC Berkeley Drive path to where I'm storing files for this semester's class. You'll want to edit that for your Drive.

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
# CHANGE THIS TO THE PATH IN YOUR OWN DRIVE WHERE YOU WANT TO SAVE CHECKPOINTS

!ls drive/MyDrive/ISchool/MIDS/266/model_checkpoints/

t5base-finetuned-shakespeare-to-modern	weights.02-0.54.hdf5  weights.05-0.54.hdf5
t5_shakespeare_weights.01-0.85.hdf5	weights.03-0.54.hdf5
weights.01-0.53.hdf5			weights.04-0.58.hdf5


Keras provides a handy [ModelCheckpoint](https://keras.io/api/callbacks/model_checkpoint/) class that we can pass into .fit as a callback. By default, it'll save a checkpoint of the model at the end of each epoch of training.

We can choose to save the whole model or just the weights (i.e. the model parameters that we've trained so far). And we'll specify the destination filepath (we can include formatting options to have different filenames for each epoch and loss).

(Other options: you can also choose to only save the best performing model each time, based on a performance metric you choose. Here we'll save after every epoch here so that you can see the resulting files as you go, but feel free to choose different options based on the ModelCheckpoint documentation if you prefer.)

In [44]:
# CHANGE checkpoint_dir TO THE PATH IN YOUR OWN DRIVE WHERE YOU WANT TO SAVE CHECKPOINTS

checkpoint_dir = 'drive/MyDrive/ISchool/MIDS/266/model_checkpoints/'
checkpoint_filepath = checkpoint_dir + 'weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

Now we're ready to train our model. We'll call `model.fit`, but instead of passing in an array of data, we'll pass in our data generator. And we'll include the model checkpoint callback, to save the weights after each epoch.

The next cell may take a couple hours to run per epoch on the full dataset. The ETA in the running output can be very useful to estimate how long it will take your model to train (and whether you need to interrupt it and make adjustments to be able to make progress).

In [45]:
model.fit(train_data_generator, validation_data=dev_data, epochs=5,
          callbacks=[model_checkpoint_callback])

Epoch 1/5


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 17/100 [====>.........................] - ETA: 48s - loss: 1.5493 - accuracy: 0.3419

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 58/100 [================>.............] - ETA: 23s - loss: 1.2387 - accuracy: 0.3685

...

 76/100 [=====================>........] - ETA: 14s - loss: 0.7106 - accuracy: 0.7007

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 84/100 [========================>.....] - ETA: 9s - loss: 0.7024 - accuracy: 0.7065 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


100/100 [==============================] - 73s 729ms/step - loss: 0.6965 - accuracy: 0.7084 - val_loss: 0.6490 - val_accuracy: 0.7360


If we need to pick up where we left off, we can load the last weights that we saved into our model and then call `model.fit` again. You'll have to edit the code line below to load the specific weight file you want to continue using.

The example below shows a filename "weights.05-0.75.hdf5". You can see the formating options we used to define checkpoint_filepath two cells above. The .05 means it was saved after the 5th epoch of training, and the 0.75 means it had validation accuracy of 75%.

Including the validation accuracy in the filename is a handy way to glance through your saved model checkpoints and make sure you're picking up using the weights that had the highest validation accuracy so far, not necessarily the last epoch.

In [46]:
# CHANGE checkpoint_filepath TO EXACT NAME OF A SAVED CHECKPOINT YOU WANT TO LOAD

checkpoint_filepath = checkpoint_dir + 'weights.05-0.74.hdf5'
model.load_weights(checkpoint_filepath)

Option 2: What if our data is stored in many small files? We might want to only load one file of data at a time, and randomly shuffle the order in which we load files from the data folder for each training epoch.

Just to demonstrate that option, we'll simulate having our data in multiple files. We'll use the same dataset, but read the full dataset once and write it to a bunch of csv files of 256 rows each. (You won't typically do this if your data starts out all in one file, in which case you can just read select rows like we did above.)

In [47]:
!mkdir SNLI_Corpus/train_files/

In [48]:
df_train = pd.read_csv(train_filename)
for i in range(0, 550152, 256):
    df_train[i:i+256].to_csv('SNLI_Corpus/train_files/train_data_%d.csv' % i, index=False)

df_train = None

In [49]:
data_dir = 'SNLI_Corpus/train_files/'
data_filenames = os.listdir(data_dir)
len(data_filenames)

2150

Now we have 2150 separate csv files of training data, and we'll only want to load one or a few at a time to train our model. Our data files have 256 rows in each, and we'll only use 32 examples per batch, so we'll usually only load one file at a time and tokenize part of it for the next batch of data.

The code below will work whether your files are larger or smaller than one batch, though. We'll keep track of which rows we've already used from the current file and take the next rows for a new batch, so we might run past the current file and load another file to fill up the rest of the batch.

In your own project, you might have data files that are smaller or larger, so we've made the code somewhat flexible so that you can see how to load just enough files to get the next batch of data that you need.

In [50]:
class SNLIDataGeneratorFromDir(tf.keras.utils.Sequence):

    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_dir,
                 examples_per_file,
                 max_length=128,
                 batch_size=32,
                 shuffle=True):

        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_dir = data_dir
        self.examples_per_file = examples_per_file
        self.max_length = max_length
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.filename_order = os.listdir(self.data_dir)
        self.next_file_i = 0
        self.next_row_i = 0

        # Call on_epoch_end to shuffle data at start
        self.on_epoch_end()

    def __len__(self):
        # Return the number of batches in the full dataset
        # return self.n_examples // self.batch_size
        return 100

    def __getitem__(self, idx):
        files_to_load = (self.batch_size // self.examples_per_file) + 1

        sentence_pairs = []
        labels = []

        for file_i in range(self.next_file_i, self.next_file_i + files_to_load):
            filepath = os.path.join(self.data_dir, self.filename_order[file_i])
            df = pd.read_csv(filepath)
            n_remaining = self.batch_size - len(sentence_pairs)

            start = self.next_row_i
            end = self.next_row_i + n_remaining
            curr_sent_pairs = df[['sentence1', 'sentence2']].values[start:end]
            sentence_pairs.extend(curr_sent_pairs.tolist())

            curr_labels = df['similarity'].values[start:end]
            labels.extend(curr_labels.tolist())

            if end < len(df):
                self.next_file_i = file_i
                self.next_row_i = end
            else:
                self.next_file_i = file_i + 1
                self.next_row_i = 0

            if len(sentence_pairs) >= self.batch_size:
                break

        batch_data = preprocess_data(
            sentence_pairs,
            labels,
            self.tokenizer,
            self.max_length
        )
        return batch_data

    def on_epoch_end(self):
        self.next_file_i = 0
        self.next_row_i = 0

        if self.shuffle:
            self.filename_order = np.random.permutation(self.filename_order)

The rest of the code is the same (with the extra "examples_per_file" paramter for this version of the data generator). Create a new data generator, and pass it into your model's .fit method. You can use the same checkpoint callback as in the first option, and load the last saved weights in the same way as well.

In [51]:
train_data_generator = SNLIDataGeneratorFromDir(
    tokenizer=bert_tokenizer,
    n_examples=550152,
    data_dir=data_dir,
    examples_per_file=256
)

In [53]:
model.fit(train_data_generator, validation_data=dev_data, epochs=5,
          callbacks=[model_checkpoint_callback])

Epoch 1/5
 60/100 [=================>............] - ETA: 13s - loss: 0.7493 - accuracy: 0.6839

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

 61/100 [=================>............] - ETA: 12s - loss: 0.7484 - accuracy: 0.6834

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


100/100 [==============================] - 46s 457ms/step - loss: 0.7303 - accuracy: 0.6913 - val_loss: 0.6510 - val_accuracy: 0.7470
Epoch 2/5
 89/100 [=========================>....] - ETA: 3s - loss: 0.6931 - accuracy: 0.7114

...

100/100 [==============================] - 51s 512ms/step - loss: 0.6900 - accuracy: 0.7138 - val_loss: 0.6121 - val_accuracy: 0.7450
Epoch 3/5
 78/100 [======================>.......] - ETA: 7s - loss: 0.7121 - accuracy: 0.6991

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


100/100 [==============================] - 46s 463ms/step - loss: 0.7015 - accuracy: 0.7053 - val_loss: 0.5966 - val_accuracy: 0.7580
Epoch 4/5
100/100 [==============================] - 45s 454ms/step - loss: 0.6995 - accuracy: 0.7066 - val_loss: 0.5996 - val_accuracy: 0.7470
Epoch 5/5
 34/100 [=========>....................] - ETA: 22s - loss: 0.6771 - accuracy: 0.7243

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

100/100 [==============================] - 46s 455ms/step - loss: 0.6921 - accuracy: 0.7181 - val_loss: 0.6074 - val_accuracy: 0.7540
